In [ ]:
import math

class ChannelThread(threading.Thread):
    def __init__(self, img_channel, windows, X_scaler):
        threading.Thread.__init__(self)
        
        self.img_channel = img_channel
        self.X_scaler = X_scaler
        self.windows = windows
        self.window_spatial = []
        self.window_hist = []
        self.window_hogs = []

    def block_from_pix(self, pix, n_blocks):
        cell_no = pix // params['pix_per_cell']
        block_no = cell_no // params['cell_per_block']

        if block_no > n_blocks:
            block_no = n_blocks - 1
        return block_no

    def get_hog_dimensions(self, width, height, pix_per_cell = 16, cell_per_block = 2, orient = 12):
        n_cells_x = width // pix_per_cell
        n_cells_y = height // pix_per_cell
        n_blocks_x = n_cells_x - cell_per_block + 1
        n_blocks_y = n_cells_y - cell_per_block + 1
        return (n_blocks_y, n_blocks_x, cell_per_block, cell_per_block, orient)
    
    def run(self):
        self.hog_features = get_hog_features(
            self.img_channel, 
            orient = params['orient'], 
            pix_per_cell = params['pix_per_cell'], 
            cell_per_block = params['cell_per_block'],  
            vis=False, 
            feature_vec=False)
        
        hog_64 = self.get_hog_dimensions(64, 64, params['pix_per_cell'], params['cell_per_block'], params['orient'])
        hog_length_64 = hog_64[0] * hog_64[1] * hog_64[2] * hog_64[3] * hog_64[4]

        pix_per_block = params['cell_per_block'] ** 2 * params['pix_per_cell']
        
        for window in self.windows:
            sub_w = window[1][0] - window[0][0]
            sub_h = window[1][1] - window[0][1]

            sub_n_cells_x = sub_w // params['pix_per_cell']
            sub_n_cells_y = sub_h // params['pix_per_cell']
            sub_n_blocks_x = sub_n_cells_x - params['cell_per_block'] + 1
            sub_n_blocks_y = sub_n_cells_y - params['cell_per_block']  + 1

            start_block_x = self.block_from_pix(window[0][0], self.hog_features.shape[1])
            start_block_y = self.block_from_pix(window[0][1] - 380, self.hog_features.shape[0])
            end_block_x = start_block_x + sub_n_blocks_x 
            end_block_y = start_block_y + sub_n_blocks_x 

            sub_hog = self.hog_features[start_block_y:end_block_y, start_block_x:end_block_x,:,:,:]
    
            pixels = sub_hog.ravel()
            cols = pixels.shape[0] / hog_length_64

            if (pixels.shape[0] % hog_length_64) > 0:
                rows_required = (hog_length_64 - 1)
                cols = pixels.shape[0] // rows_required
                ok_pixels = rows_required * cols

                sub_pixels_for_mean = pixels[:good_features].reshape(-1, cols)
                w_hog = np.mean(sub_pixels_for_mean, axis=1)
                
                # this window hog would be one row short, so lets
                # add the mean of whatever pixels are left to this
                w_hog = np.hstack((w_hog, np.mean(pixels[good_features:])))
            else:
                cols = pixels.shape[0] // hog_length_64
                sub_pixels = pixels.reshape(-1, cols)
                w_hog = np.mean(sub_pixels, axis=1)
            
            self.window_hogs.append(w_hog)
            
            window_img = self.img_cs[window[0][1]:window[1][1], window[0][0]:window[1][0]]
            
            color = cv2.resize(window_img, params['spatial_size']).ravel()
            self.window_spatial.append(color)
            
            self.window_hist = np.histogram(window_img, bins=params['hist_bins'])

class SlidingWindowFeatureFinder():
    def __init__(self, svc, X_scaler):
        self.img_cs = None
        self.windows = None
        self.svc = svc
        self.X_scaler = X_scaler
        self.window_features = []
        
    # returns features for all windows
    def find_features(self, img_cs):
        if self.windows is None:
            self.windows = get_all_windows(img_cs)
            
        threads = []

        t1 = time.time()
        for channel in range(3):
            thread = ChannelThread(img_cs[:,:,channel], self.windows, X_scaler)
            threads.append(thread)
            thread.start()

        self.window_features = []
        for thread in threads:
            thread.join()
            
        self.window_features = np.hstack(
            (thread[0].window_spatial, thread[1].window_spatial, thread[2].window_spatial,
            thread[0].window_hist, thread[1].window_hist, thread[2].window_hist,
            thread[0].window_hogs, thread[1].window_hogs, thread[2].window_hogs))

        t2 = time.time()
        print('Total time taken for {} boxes to be reduced and hogged {:.2f} secs'.format(len(self.windows), t2 - t1))
        print('Total window features:', len(self.window_features))
        print('Window Feature Length:', self.window_features[0].shape)
    
    def predict_cars(self, img_cs):
        self.find_features(img_cs)
        
#         for window in self.window_features:
#             if self.svc.predict(window):
#                 print('found')
        
img_cs = load_image('./project_video-frames/0000.jpg', params['color_space'])
wf = SlidingWindowFeatureFinder(svc, X_scaler)
wf.predict_cars(img_cs)